# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-16-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-16-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-17 05:22:17,33.93911,67.709953,53938,2336,45465,6137.0,Afghanistan,138.557150,4.330898
1,NaN,NaN,NaN,Albania,2021-01-17 05:22:17,41.15330,20.168300,67216,1270,40090,25856.0,Albania,2335.673084,1.889431
2,NaN,NaN,NaN,Algeria,2021-01-17 05:22:17,28.03390,1.659600,103611,2831,70373,30407.0,Algeria,236.279443,2.732335
3,NaN,NaN,NaN,Andorra,2021-01-17 05:22:17,42.50630,1.521800,9038,91,8116,831.0,Andorra,11697.405035,1.006860
4,NaN,NaN,NaN,Angola,2021-01-17 05:22:17,-11.20270,17.873900,18765,431,16225,2109.0,Angola,57.095013,2.296829


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2253,2257,2264,2277,2288,2301,2301,2314,2324,2336
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1223,1230,1233,1241,1247,1252,1256,1261,1265,1270
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2792,2798,2803,2807,2812,2816,2819,2822,2827,2831


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,43291,43440,43740,43948,44137,44608,44850,45298,45434,45465
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,36535,36971,37327,37648,37981,38421,38860,39246,39625,40090
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,68589,68802,69011,69212,69403,69608,69791,69992,70188,70373


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4705,4770,4847,4879,4902,4970,4998,5075,5103,5154
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,14845,15052,15202,15327,15417,15572,15701,15841,16002,16176
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1614,1634,1648,1658,1663,1679,1685,1696,1712,1723


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,50,50,53,54,55,55,55,55,55,55
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,171,171,173,173,173,175,175,177,179,182
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,35,35,35,35,35,35,35,36,36,36


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
650,1001.0,Autauga,Alabama,US,2021-01-17 05:22:17,32.539527,-86.644082,5154,55,0,5099.0,"Autauga, Alabama, US",9225.151694,1.067132
687,1075.0,Lamar,Alabama,US,2021-01-17 05:22:17,33.779950,-88.096680,1175,19,0,1156.0,"Lamar, Alabama, US",8511.408910,1.617021
688,1077.0,Lauderdale,Alabama,US,2021-01-17 05:22:17,34.901719,-87.656247,7528,83,0,7445.0,"Lauderdale, Alabama, US",8118.280150,1.102550


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,23754315,395785,0
India,10557985,152274,10196885
Brazil,8455059,209296,7485823


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,23754315,395785,0,23358530,2021-01-17 05:22:17,37.936303,-91.379001
India,10557985,152274,10196885,208826,2021-01-17 05:22:17,23.088275,81.806127
Brazil,8455059,209296,7485823,759940,2021-01-17 05:22:17,-12.669522,-48.480493
Russia,3507201,64134,2905972,537095,2021-01-17 05:22:17,54.546312,62.120860
United Kingdom,3367070,88747,7862,3270461,2021-01-17 05:22:17,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2955323,33342,0
Texas,2112662,32428,0
Florida,1560186,24004,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2955323,33342,0,2921981,2021-01-17 05:22:17,37.843962,-120.728594
Texas,2112662,32428,0,2080234,2021-01-17 05:22:17,31.660643,-98.653069
Florida,1560186,24004,0,1536182,2021-01-17 05:22:17,28.940755,-82.700744
New York,1228867,40806,0,1188061,2021-01-17 05:22:17,42.544151,-75.474183
Illinois,1064667,20020,0,1044647,2021-01-17 05:22:17,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1003923,13741,0
Illinois,Cook,431385,8980,0
Arizona,Maricopa,413282,6437,0
Florida,Miami-Dade,341842,4561,0
Texas,Harris,281422,3777,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1003923,13741,0,990182,2021-01-17 05:22:17,34.308284,-118.228241,6037.0
Illinois,Cook,431385,8980,0,422405,2021-01-17 05:22:17,41.841448,-87.816588,17031.0
Arizona,Maricopa,413282,6437,0,406845,2021-01-17 05:22:17,33.348359,-112.491815,4013.0
Florida,Miami-Dade,341842,4561,0,337281,2021-01-17 05:22:17,25.611236,-80.551706,12086.0
Texas,Harris,281422,3777,0,277645,2021-01-17 05:22:17,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,53775,65994,103127,8868,18613,184,1770715,163576,28669,389260,...,3269757,29989,77777,1,118415,1531,150505,2110,32800,25368
2021-01-15,53831,66635,103381,8946,18679,184,1783047,163972,28689,390788,...,3325642,30946,77904,1,118856,1536,151142,2111,34278,26109
2021-01-16,53938,67216,103611,9038,18765,187,1791979,164235,28708,392511,...,3367070,31669,77968,1,119306,1537,151569,2112,36074,26881


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,2314,1261,2822,88,425,6,45125,2963,909,6921,...,86163,280,618,0,1090,35,1665,612,514,636
2021-01-15,2324,1265,2827,88,428,6,45227,2974,909,6987,...,87448,291,619,0,1095,35,1687,612,527,666
2021-01-16,2336,1270,2831,91,431,6,45295,2987,909,7053,...,88747,298,619,0,1101,35,1700,612,537,683


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,45298,39246,69992,8091,16008,156,1549490,151849,25884,362931,...,7580,21685,76276,1,111999,1369,136247,1419,22504,14714
2021-01-15,45434,39625,70188,8116,16120,156,1561963,152320,25897,365214,...,7691,22423,76388,1,112320,1380,137648,1419,24105,15414
2021-01-16,45465,40090,70373,8116,16225,156,1573104,152615,25913,367785,...,7862,23136,76436,1,112622,1380,138495,1419,25106,15872


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4705,4770,4847,4879,4902,4970,4998,5075,5103,5154
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1114,1124,1132,1137,1146,1153,1157,1168,1166,1175
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,6930,7051,7149,7191,7214,7296,7350,7402,7451,7528
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2217,2263,2315,2324,2340,2351,2368,2393,2404,2425
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,11179,11355,11498,11565,11626,11812,11983,12120,12261,12393


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-14,414583,51157,0,649040,264511,2883699,369177,221544,68065,49,...,670482,2068637,317559,9573,2166,417839,283777,105219,561160,48701
2021-01-15,417528,51466,0,658186,267635,2922690,371458,223422,68727,49,...,676039,2094921,320102,9734,2166,422634,285970,106649,563866,48909
2021-01-16,420681,51653,0,666901,270178,2955323,373483,223422,69372,49,...,680847,2112662,322252,9915,2166,429391,285970,108124,566275,49068


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-14        5075   15841    1696  2113   5219     971   1669   10863   
2021-01-15        5103   16002    1712  2130   5264     981   1679   10982   
2021-01-16        5154   16176    1723  2144   5292     987   1684   11078   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-14         2757     1583  ...    2368    509     2740      632   
2021-01-15         2778     1596  ...    2396    515     2746      633   
2021-01-16         2818     1600  ...    2400    516     2763      634   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-14           3250  2579  1794          0      837    602  
2021-01-15           3262  2605  1806          0      840    602  
2021-01-16           3280  2614  1813          0      842    602  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-14,5945,228,0,10855,4228,32318,5316,6553,995,0,...,8232,31693,1460,162,24,5626,3876,1702,5728,522
2021-01-15,6030,229,0,11040,4263,33002,5343,6594,1002,0,...,8311,32086,1472,163,24,5656,3903,1733,5770,522
2021-01-16,6119,229,0,11248,4293,33342,5363,6594,1016,0,...,8355,32428,1485,163,24,5706,3903,1761,5905,522


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-14          55     177      36   47     80      28     58     197   
2021-01-15          55     179      36   47     83      28     58     200   
2021-01-16          55     182      36   47     83      28     58     200   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-14           69       30  ...      16      9       17        7   
2021-01-15           69       30  ...      16      9       17        7   
2021-01-16           69       30  ...      16      9       17        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-14             26     4     9          0       23      4  
2021-01-15             26     4     9          0       23      4  
2021-01-16             26     4     9          0       23      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,0.003564,0.010102,0.002596,0.005670,0.010204,0.045455,0.007560,0.002746,0.000314,0.009099,...,0.015152,0.053169,0.000785,0.0,0.005127,0.006575,0.004914,0.000474,0.054662,0.045844
2021-01-15,0.001041,0.009713,0.002463,0.008796,0.003546,0.000000,0.006964,0.002421,0.000698,0.003925,...,0.017091,0.031912,0.001633,0.0,0.003724,0.003266,0.004232,0.000474,0.045061,0.029210
2021-01-16,0.001988,0.008719,0.002225,0.010284,0.004604,0.016304,0.005009,0.001604,0.000662,0.004409,...,0.012457,0.023363,0.000822,0.0,0.003786,0.000651,0.002825,0.000474,0.052395,0.029568


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,0.005650,0.003981,0.001064,0.011494,0.002358,0.2,0.003157,0.004066,0.0,0.007717,...,0.014757,0.018182,0.000000,NaN,0.005535,0.0,0.004222,0.0,0.009823,0.079796
2021-01-15,0.004322,0.003172,0.001772,0.000000,0.007059,0.0,0.002260,0.003712,0.0,0.009536,...,0.014914,0.039286,0.001618,NaN,0.004587,0.0,0.013213,0.0,0.025292,0.047170
2021-01-16,0.005164,0.003953,0.001415,0.034091,0.007009,0.0,0.001504,0.004371,0.0,0.009446,...,0.014855,0.024055,0.000000,NaN,0.005479,0.0,0.007706,0.0,0.018975,0.025526


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,0.009989,0.009933,0.002880,0.002602,0.024119,0.026316,0.008505,0.003987,0.000348,0.009005,...,0.020188,0.045917,0.001037,0.0,0.003431,0.000000,0.009409,0.0,0.043398,0.077317
2021-01-15,0.003002,0.009657,0.002800,0.003090,0.006997,0.000000,0.008050,0.003102,0.000502,0.006290,...,0.014644,0.034033,0.001468,0.0,0.002866,0.008035,0.010283,0.0,0.071143,0.047574
2021-01-16,0.000682,0.011735,0.002636,0.000000,0.006514,0.000000,0.007133,0.001937,0.000618,0.007040,...,0.022234,0.031798,0.000628,0.0,0.002689,0.000000,0.006153,0.0,0.041527,0.029713


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-14,0.008730,0.006710,NaN,0.011393,0.009507,0.014293,0.006552,0.004389,0.013279,0.0,...,0.007488,0.013665,0.008710,0.021883,0.0,0.012833,0.009157,0.007922,0.005627,0.008532
2021-01-15,0.007104,0.006040,NaN,0.014092,0.011810,0.013521,0.006179,0.008477,0.009726,0.0,...,0.008288,0.012706,0.008008,0.016818,0.0,0.011476,0.007728,0.013591,0.004822,0.004271
2021-01-16,0.007552,0.003633,NaN,0.013241,0.009502,0.011165,0.005451,0.000000,0.009385,0.0,...,0.007112,0.008469,0.006717,0.018595,0.0,0.015988,0.000000,0.013830,0.004272,0.003251


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-14      0.015406  0.008917  0.006528  0.001897  0.009478  0.005176   
2021-01-15      0.005517  0.010163  0.009434  0.008045  0.008622  0.010299   
2021-01-16      0.009994  0.010874  0.006425  0.006573  0.005319  0.006116   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-14      0.010902  0.010982  0.008413  0.008923  ...  0.009378   
2021-01-15      0.005992  0.010955  0.007617  0.008212  ...  0.011824   
2021-01-16      0.002978  0.008742  0.014399  0.002506  ...  0.001669   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-14      0.018000  0.006983  0.001585   0.008690  0.030775  0.011274   
2021-01-15      0.011788  0.002190  0.001582   0.003692  0.010081  0.006689   
2021-01-16      0.001942  0.006191  0.001580   0.005518  0.003455  0.003876   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-14            NaN  0.008434  0.001664  
2021-01-15            NaN  0.003584  0.000000  
2021-01-16            NaN  0.002381  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-14,0.032118,0.008850,NaN,0.017052,0.010033,0.020010,0.005866,0.002601,0.001006,NaN,...,0.010309,0.013301,0.007591,0.025316,0.0,0.013329,0.009901,0.018552,0.008628,0.0
2021-01-15,0.014298,0.004386,NaN,0.017043,0.008278,0.021165,0.005079,0.006257,0.007035,NaN,...,0.009597,0.012400,0.008219,0.006173,0.0,0.005332,0.006966,0.018214,0.007332,0.0
2021-01-16,0.014760,0.000000,NaN,0.018841,0.007037,0.010302,0.003743,0.000000,0.013972,NaN,...,0.005294,0.010659,0.008832,0.000000,0.0,0.008840,0.000000,0.016157,0.023397,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                \
Admin2         Autauga   Baldwin   Barbour      Bibb  Blount Bullock   
2021-01-14         0.0  0.011429  0.028571 -0.020833  0.0000     0.0   
2021-01-15         0.0  0.011299  0.000000  0.000000  0.0375     0.0   
2021-01-16         0.0  0.016760  0.000000  0.000000  0.0000     0.0   

Province_State                                          ... Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park Platte   
2021-01-14      0.017544  0.000000  0.045455  0.034483  ...     0.0    0.0   
2021-01-15      0.000000  0.015228  0.000000  0.000000  ...     0.0    0.0   
2021-01-16      0.000000  0.000000  0.000000  0.000000  ...     0.0    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2021-01-14          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-01-15          0.0      0.0        0.0   0.0   0.0        NaN      0.0   
2021-01-16          0.0      0.0        0.0   0.0   0.0        NaN      0.0   

Province_State         
Admin2         Weston  
2021-01-14        0.0  
2021-01-15        0.0  
2021-01-16        0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,0.002061,0.010049,0.002452,0.008482,0.007262,0.024439,0.007280,0.002584,0.000394,0.006889,...,0.015322,0.040699,0.000717,0.0,0.004424,0.003973,0.005116,0.000628,0.048360,0.044481
2021-01-15,0.001551,0.009881,0.002458,0.008639,0.005404,0.012219,0.007122,0.002502,0.000546,0.005407,...,0.016207,0.036305,0.001175,0.0,0.004074,0.003619,0.004674,0.000551,0.046710,0.036846
2021-01-16,0.001769,0.009300,0.002341,0.009461,0.005004,0.014262,0.006066,0.002053,0.000604,0.004908,...,0.014332,0.029834,0.000998,0.0,0.003930,0.002135,0.003750,0.000512,0.049553,0.033207


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,0.004108,0.003922,0.001192,0.010311,0.003656,0.100,0.003257,0.003554,2.100604e-09,0.007643,...,0.015041,0.021577,0.000472,NaN,0.005587,1.826162e-42,0.008893,0.000154,0.020224,0.067907
2021-01-15,0.004215,0.003547,0.001482,0.005155,0.005357,0.050,0.002759,0.003633,1.050302e-09,0.008590,...,0.014977,0.030432,0.001045,NaN,0.005087,9.130809e-43,0.011053,0.000077,0.022758,0.057538
2021-01-16,0.004689,0.003750,0.001448,0.019623,0.006183,0.025,0.002131,0.004002,5.251511e-10,0.009018,...,0.014916,0.027243,0.000522,NaN,0.005283,4.565404e-43,0.009380,0.000038,0.020867,0.041532


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-14,0.008307,0.010440,0.002825,0.009177,0.026937,0.013432,0.007049,0.004165,0.000411,0.007426,...,0.019018,0.038337,0.000944,0.0,0.003580,0.001561,0.009709,0.000961,0.030376,0.049249
2021-01-15,0.005655,0.010048,0.002813,0.006134,0.016967,0.006716,0.007549,0.003633,0.000457,0.006858,...,0.016831,0.036185,0.001206,0.0,0.003223,0.004798,0.009996,0.000481,0.050759,0.048411
2021-01-16,0.003168,0.010892,0.002724,0.003067,0.011740,0.003358,0.007341,0.002785,0.000537,0.006949,...,0.019532,0.033991,0.000917,0.0,0.002956,0.002399,0.008075,0.000240,0.046143,0.039062


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-14,0.008417,0.005726,NaN,0.011664,0.009802,0.014695,0.006216,0.010896,0.011898,1.044488e-92,...,0.007169,0.012915,0.010295,0.018634,0.001495,0.012213,0.009369,0.009645,0.005342,0.008163
2021-01-15,0.007760,0.005883,NaN,0.012878,0.010806,0.014108,0.006197,0.009686,0.010812,5.222440e-93,...,0.007729,0.012810,0.009152,0.017726,0.000747,0.011844,0.008548,0.011618,0.005082,0.006217
2021-01-16,0.007656,0.004758,NaN,0.013059,0.010154,0.012637,0.005824,0.004843,0.010098,2.611220e-93,...,0.007420,0.010639,0.007934,0.018160,0.000374,0.013916,0.004274,0.012724,0.004677,0.004734


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-14      0.011855  0.008781  0.006068  0.005948  0.008386  0.004239   
2021-01-15      0.008686  0.009472  0.007751  0.006997  0.008504  0.007269   
2021-01-16      0.009340  0.010173  0.007088  0.006785  0.006912  0.006693   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-14      0.007269  0.009529  0.010724  0.007371  ...  0.011340   
2021-01-15      0.006630  0.010242  0.009171  0.007792  ...  0.011582   
2021-01-16      0.004804  0.009492  0.011785  0.005149  ...  0.006626   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-14      0.014211  0.005621  0.003531   0.006916  0.021392  0.010358   
2021-01-15      0.013000  0.003905  0.002556   0.005304  0.015737  0.008523   
2021-01-16      0.007471  0.005048  0.002068   0.005411  0.009596  0.006200   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-14           -1.0  0.008212  0.030796  
2021-01-15           -1.0  0.005898  0.015398  
2021-01-16           -1.0  0.004139  0.007699  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-14,0.030699,0.006248,NaN,0.018064,0.011360,0.018939,0.006138,0.006331,0.003176,NaN,...,0.013146,0.011743,0.011447,0.014481,0.000021,0.012484,0.011465,0.019326,0.007986,0.008707
2021-01-15,0.022498,0.005317,NaN,0.017553,0.009819,0.020052,0.005608,0.006294,0.005105,NaN,...,0.011371,0.012072,0.009833,0.010327,0.000011,0.008908,0.009215,0.018770,0.007659,0.004353
2021-01-16,0.018629,0.002658,NaN,0.018197,0.008428,0.015177,0.004676,0.003147,0.009539,NaN,...,0.008333,0.011365,0.009332,0.005163,0.000005,0.008874,0.004608,0.017463,0.015528,0.002177


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-14      0.002687  0.007369  0.014406 -0.010330  0.007570  0.002134   
2021-01-15      0.001344  0.009334  0.007203 -0.005165  0.022535  0.001067   
2021-01-16      0.000672  0.013047  0.003601 -0.002583  0.011268  0.000533   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-14      0.013009  0.017310  0.024216  0.043236  ...  0.009865   
2021-01-15      0.006504  0.016269  0.012108  0.021618  ...  0.004932   
2021-01-16      0.003252  0.008135  0.006054  0.010809  ...  0.002466   

Province_State                                                         \
Admin2            Platte      Sheridan  Sublette Sweetwater     Teton   
2021-01-14      0.000122  2.161949e-06  0.000169   0.011630  0.000012   
2021-01-15      0.000061  1.080975e-06  0.000084   0.005815  0.000006   
2021-01-16      0.000031  5.404873e-07  0.000042   0.002907  0.000003   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2021-01-14      0.016183  -0.501961  0.012015  0.001790  
2021-01-15      0.008092  -0.501961  0.006007  0.000895  
2021-01-16      0.004046  -0.501961  0.003004  0.000448  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210116,AK,50017,NaN,1348632.0,NaN,totalTestsViral,1398649,67.0,1152.0,...,1398649,0,0,c8702a5885696c80f7ada7729246c5e8a7e80190,0,0,0,0,0,NaN
1,20210116,AL,420681,86238.0,1686680.0,NaN,totalTestsPeopleViral,2021123,2772.0,38763.0,...,2107361,89,0,229775a467e5fa5612f8104e0e1e6d26eacf9fde,0,0,0,0,0,NaN
2,20210116,AR,270178,52130.0,2043157.0,NaN,totalTestsViral,2261205,1292.0,12686.0,...,2313335,30,88,3da73e1d161f611da26e52dd0430b0c9f968d8fb,0,0,0,0,0,NaN
3,20210116,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,403b1c6fc78bb3702ab76d583a77bb4d9137d230,0,0,0,0,0,NaN
4,20210116,AZ,666901,36906.0,2563182.0,NaN,totalTestsViral,5969026,4849.0,46648.0,...,3230083,208,983,c3d8f15bde218c663f3a761a9aa81c0f0aef5b48,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210116,AK,50017.0,NaN,1348632.0,NaN,totalTestsViral,1398649.0,67.0,1152.0,...,1398649,0,0,c8702a5885696c80f7ada7729246c5e8a7e80190,0,0,0,0,0,NaN
1,20210116,AL,420681.0,86238.0,1686680.0,NaN,totalTestsPeopleViral,2021123.0,2772.0,38763.0,...,2107361,89,0,229775a467e5fa5612f8104e0e1e6d26eacf9fde,0,0,0,0,0,NaN
2,20210116,AR,270178.0,52130.0,2043157.0,NaN,totalTestsViral,2261205.0,1292.0,12686.0,...,2313335,30,88,3da73e1d161f611da26e52dd0430b0c9f968d8fb,0,0,0,0,0,NaN
3,20210116,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,403b1c6fc78bb3702ab76d583a77bb4d9137d230,0,0,0,0,0,NaN
4,20210116,AZ,666901.0,36906.0,2563182.0,NaN,totalTestsViral,5969026.0,4849.0,46648.0,...,3230083,208,983,c3d8f15bde218c663f3a761a9aa81c0f0aef5b48,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-16,AK,50017,NaN,1348632.0,NaN,totalTestsViral,1398649,67.0,1152.0,NaN,...,1398649,0,0,c8702a5885696c80f7ada7729246c5e8a7e80190,0,0,0,0,0,NaN
2021-01-16,AL,420681,86238.0,1686680.0,NaN,totalTestsPeopleViral,2021123,2772.0,38763.0,NaN,...,2107361,89,0,229775a467e5fa5612f8104e0e1e6d26eacf9fde,0,0,0,0,0,NaN
2021-01-16,AR,270178,52130.0,2043157.0,NaN,totalTestsViral,2261205,1292.0,12686.0,421.0,...,2313335,30,88,3da73e1d161f611da26e52dd0430b0c9f968d8fb,0,0,0,0,0,NaN
2021-01-16,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,403b1c6fc78bb3702ab76d583a77bb4d9137d230,0,0,0,0,0,NaN
2021-01-16,AZ,666901,36906.0,2563182.0,NaN,totalTestsViral,5969026,4849.0,46648.0,1144.0,...,3230083,208,983,c3d8f15bde218c663f3a761a9aa81c0f0aef5b48,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-16,AK,50017.0,NaN,1348632.0,NaN,totalTestsViral,1398649.0,67.0,1152.0,NaN,...,1398649,0,0,c8702a5885696c80f7ada7729246c5e8a7e80190,0,0,0,0,0,NaN
2021-01-16,AL,420681.0,86238.0,1686680.0,NaN,totalTestsPeopleViral,2021123.0,2772.0,38763.0,NaN,...,2107361,89,0,229775a467e5fa5612f8104e0e1e6d26eacf9fde,0,0,0,0,0,NaN
2021-01-16,AR,270178.0,52130.0,2043157.0,NaN,totalTestsViral,2261205.0,1292.0,12686.0,421.0,...,2313335,30,88,3da73e1d161f611da26e52dd0430b0c9f968d8fb,0,0,0,0,0,NaN
2021-01-16,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,403b1c6fc78bb3702ab76d583a77bb4d9137d230,0,0,0,0,0,NaN
2021-01-16,AZ,666901.0,36906.0,2563182.0,NaN,totalTestsViral,5969026.0,4849.0,46648.0,1144.0,...,3230083,208,983,c3d8f15bde218c663f3a761a9aa81c0f0aef5b48,0,0,0,0,0,NaN
